In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import time
pd.set_option('display.max_columns', None)

In [12]:
# Load the large dataset
print("Loading large dataset...")
start_time = time.time()
df_large = pd.read_json("../src/data/customer_churn.json", lines=True)
print(f"Loaded {len(df_large):,} records in {time.time() - start_time:.2f} seconds")

Loading large dataset...
Loaded 543,705 records in 2.04 seconds


In [13]:
df_large.head()

,ts,userId,sessionId,page,auth,method,status,level,itemInSession,location,userAgent,lastName,firstName,registration,gender,artist,song,length
0,1538352011000,293,292,NextSong,Logged In,PUT,200,free,20,"Corpus Christi, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Morales,Joseph,1.532064e+12,M,Martin Orford,Grand Designs,597.55057
1,1538352025000,98,97,NextSong,Logged In,PUT,200,free,74,"Houston-The Woodlands-Sugar Land, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Larson,Sawyer,1.538070e+12,M,John Brown's Body,Bulls,380.21179
2,1538352118000,179,178,NextSong,Logged In,PUT,200,paid,184,"Orlando-Kissimmee-Sanford, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Santiago,Maverick,1.535953e+12,M,Afroman,Because I Got High,202.37016
3,1538352119000,179,178,Logout,Logged In,PUT,307,paid,185,"Orlando-Kissimmee-Sanford, FL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Santiago,Maverick,1.535953e+12,M,NaN,NaN,NaN
4,1538352124000,246,245,NextSong,Logged In,PUT,200,paid,22,"Mobile, AL",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,Campos,Gianna,1.535931e+12,F,Lily Allen,Smile (Radio Edit),194.53342


In [14]:
def is_valid_sequence_assignment(user_items_array, item):
    """Check if assigning an item to a user creates a valid consecutive sequence."""
    if len(user_items_array) == 0:
        return True
    
    min_item = user_items_array[0]
    max_item = user_items_array[-1]
    
    if item == max_item + 1 or item == min_item - 1:
        return True
    elif min_item < item < max_item:
        for existing_item in user_items_array:
            if abs(existing_item - item) == 1:
                return True
        return False
    else:
        return False

def fill_missing_userids(df):
    """Fill missing userIDs using session sequence logic."""
    df = df.copy()
    df['imputed'] = False
    df['ts'] = pd.to_datetime(df['ts'])
    
    userId_col = df.columns.get_loc('userId')
    imputed_col = df.columns.get_loc('imputed')

    for session in df[df['userId'].isna()]['sessionId'].unique():
        mask = df['sessionId'] == session
        session_df = df[mask].copy()
        
        userId_array = session_df['userId'].values
        itemInSession_array = session_df['itemInSession'].values
        ts_array = pd.to_datetime(session_df['ts']).values
        
        max_iterations = len(session_df)
        iteration = 0
        
        while iteration < max_iterations:
            iteration += 1
            filled_any = False
            user_items_cache = {}
            
            missing_mask = pd.isna(userId_array)
            missing_indices = np.where(missing_mask)[0]
            
            if len(missing_indices) == 0:
                break
            
            for i in missing_indices:
                missing_ts = ts_array[i]
                missing_item = itemInSession_array[i]
                
                forward_user = None
                backward_user = None
                forward_ts = None
                backward_ts = None
                
                # Look forward
                for j in range(i + 1, len(userId_array)):
                    candidate_user = userId_array[j]
                    if not pd.isna(candidate_user):
                        if candidate_user not in user_items_cache:
                            user_mask = userId_array == candidate_user
                            user_items_cache[candidate_user] = np.sort(itemInSession_array[user_mask])
                        
                        user_items = user_items_cache[candidate_user]
                        
                        if missing_item not in user_items:
                            if is_valid_sequence_assignment(user_items, missing_item):
                                forward_user = candidate_user
                                forward_ts = ts_array[j]
                                break
                
                # Look backward
                for j in range(i - 1, -1, -1):
                    candidate_user = userId_array[j]
                    if not pd.isna(candidate_user):
                        if candidate_user not in user_items_cache:
                            user_mask = userId_array == candidate_user
                            user_items_cache[candidate_user] = np.sort(itemInSession_array[user_mask])
                        
                        user_items = user_items_cache[candidate_user]
                        
                        if missing_item not in user_items:
                            if is_valid_sequence_assignment(user_items, missing_item):
                                backward_user = candidate_user
                                backward_ts = ts_array[j]
                                break
                
                # Choose user based on timestamp proximity
                chosen_user = None
                
                if forward_user is not None and backward_user is not None:
                    forward_diff = abs((forward_ts - missing_ts).astype('timedelta64[s]').astype(int))
                    backward_diff = abs((backward_ts - missing_ts).astype('timedelta64[s]').astype(int))
                    
                    if forward_diff <= backward_diff:
                        chosen_user = forward_user
                    else:
                        chosen_user = backward_user
                elif forward_user is not None:
                    chosen_user = forward_user
                elif backward_user is not None:
                    chosen_user = backward_user
                
                if chosen_user is not None:
                    userId_array[i] = chosen_user
                    session_df.iloc[i, imputed_col] = True
                    filled_any = True
                    if chosen_user in user_items_cache:
                        del user_items_cache[chosen_user]
            
            if not filled_any:
                break
        
        session_df.iloc[:, userId_col] = userId_array
        df.loc[mask, 'userId'] = session_df['userId'].values
        df.loc[mask, 'imputed'] = session_df['imputed'].values

    return df

def map_user_attributes(df):
    """Map user attributes based on userId."""
    df = df.copy()
    user_cols = ['location', 'userAgent', 'lastName', 'firstName', 'registration', 'gender']
    
    user_map = df[df['userId'].notna()].groupby('userId')[user_cols].first()
    
    for col in user_cols:
        df[col] = df['userId'].map(user_map[col]).fillna(df[col])
    
    return df

In [15]:
def preprocess_data(df):
    df_preprocessed = df.copy()
    df_preprocessed['ts'] = pd.to_datetime(df_preprocessed['ts'], unit='ms')
    df_preprocessed['registration'] = pd.to_datetime(df_preprocessed['registration'], unit='ms')
    df_preprocessed = df_preprocessed.sort_values(['ts', 'sessionId', 'itemInSession']).reset_index(drop=True)
    df_preprocessed['userId'] = pd.to_numeric(df_preprocessed['userId'], errors='coerce')
    df_preprocessed = fill_missing_userids(df_preprocessed)
    df_preprocessed = map_user_attributes(df_preprocessed)
    df_preprocessed = df_preprocessed.dropna(subset=['userId']).reset_index(drop=True)
    df_preprocessed['userId'] = df_preprocessed['userId'].astype(int)
    df_preprocessed['city'] = df_preprocessed['location'].str.split(',').str[0]
    df_preprocessed['state'] = df_preprocessed['location'].str.split(',').str[1].str.strip()

    return df_preprocessed

In [16]:
dataframe_processed = preprocess_data(df_large)

In [17]:
def extract_features(df):
    demographic_features = df.groupby('userId').agg({
    'gender': 'first',
    'state': 'first'
    }).fillna('Unknown')

    # One-hot encode gender
    gender_encoded = pd.get_dummies(demographic_features['gender'], prefix='gender')

    # One-hot encode state (top 3 states + 'Other')
    state_counts = demographic_features['state'].value_counts()
    top_states = state_counts.head(3).index.tolist()
    demographic_features['state_grouped'] = demographic_features['state'].apply(
        lambda x: x if x in top_states else 'Other'
    )
    state_encoded = pd.get_dummies(demographic_features['state_grouped'], prefix='state')

    # Combine demographic features
    demographic_final = pd.concat([gender_encoded, state_encoded], axis=1)

    # Activity Features
    user_activity = df.groupby('userId').agg({
        'ts': 'count',                    # total_events
        'sessionId': 'nunique',           # num_sessions  
        'itemInSession': 'sum'            # total_interactions
    }).fillna(0)

    user_activity.columns = ['total_events', 'num_sessions', 'total_interactions']
    user_activity['events_per_session'] = (
        user_activity['total_events'] / user_activity['num_sessions']
    ).fillna(0)

    # Listening Behavior Features
    songs_df = df[df['song'].notna()]

    listening_features = songs_df.groupby('userId').agg({
        'ts': 'count',                    # songs_played
        'length': ['sum', 'mean'],        # total_listening_time, avg_song_length
        'artist': 'nunique',              # unique_artists
        'song': 'nunique',                # unique_songs
        'registration': 'first'           # registration_date
    }).fillna(0)

    listening_features.columns = ['songs_played', 'total_listening_time', 'avg_song_length', 
                                'unique_artists', 'unique_songs', 'registration_date']

    # Calculate days since registration
    max_date = df['ts'].max()
    listening_features['days_since_registration'] = (
        (max_date - pd.to_datetime(listening_features['registration_date'])).dt.total_seconds() / (24 * 3600)
    )

    # Daily averages
    listening_features['avg_daily_listening_time'] = (
        listening_features['total_listening_time'] / listening_features['days_since_registration']
    ).fillna(0)

    listening_features['avg_daily_songs'] = (
        listening_features['songs_played'] / listening_features['days_since_registration']
    ).fillna(0)

    # Artist diversity
    listening_features['artist_diversity'] = (
        listening_features['unique_artists'] / listening_features['songs_played']
    ).fillna(0)

    listening_features = listening_features.drop('registration_date', axis=1)

    # Engagement Features
    thumbs_up = df.query("page == 'Thumbs Up'").groupby('userId').size()
    thumbs_down = df.query("page == 'Thumbs Down'").groupby('userId').size()

    engagement_features = pd.DataFrame({
        'thumbs_up': thumbs_up,
        'thumbs_down': thumbs_down
    }).fillna(0)

    engagement_features['total_feedback'] = engagement_features['thumbs_up'] + engagement_features['thumbs_down']
    engagement_features['positive_feedback_ratio'] = (
        engagement_features['thumbs_up'] / engagement_features['total_feedback']
    ).fillna(0.5)

    # Other engagement metrics
    engagement_features['playlist_adds'] = df[df['page'] == 'Add to Playlist'].groupby('userId').size()
    engagement_features['add_friend'] = df[df['page'] == 'Add Friend'].groupby('userId').size()
    engagement_features['advert_roll'] = df[df['page'] == 'Roll Advert'].groupby('userId').size()
    engagement_features = engagement_features.fillna(0)

    # Subscription Features
    latest_level = df.groupby('userId')['level'].last()
    subscription_features = pd.DataFrame({
        'is_paid': (latest_level == 'paid').astype(int)
    })

    level_changes = df.groupby('userId')['level'].nunique()
    subscription_features['subscription_changes'] = (level_changes > 1).astype(int)

    subscription_features['downgrades'] = df.query("page == 'Submit Downgrade'").groupby('userId').size()
    subscription_features['upgrades'] = df.query("page == 'Submit Upgrade'").groupby('userId').size()
    subscription_features = subscription_features.fillna(0)

    # Technical Issues Features
    issue_features = pd.DataFrame({
        'error_count': df.query("page == 'Error'").groupby('userId').size(),
        'help_visits': df.query("page == 'Help'").groupby('userId').size(),
        'settings_visits': df.query("page == 'Settings'").groupby('userId').size(),
        'logout_count': df.query("page == 'Logout'").groupby('userId').size()
    }).fillna(0)

    issue_features['has_issues'] = (
        (issue_features['error_count'] > 0) | 
        (issue_features['help_visits'] > 0)
    ).astype(int)

    # Temporal Features
    last_activity = df.groupby('userId')['ts'].max()
    days_since_last_activity = (max_date - last_activity).dt.total_seconds() / (24 * 3600)

    user_registration = df.groupby('userId')['registration'].first()
    user_registration = pd.to_datetime(user_registration)

    days_available = []
    for user_id in user_registration.index:
        reg_date = user_registration[user_id]
        days_available.append((max_date.date() - reg_date.date()).days + 1)

    days_used = df.groupby('userId')['ts'].apply(lambda x: x.dt.date.nunique())

    temporal_features = pd.DataFrame({
        'days_since_last_activity': days_since_last_activity,
        'days_used_in_period': days_used,
        'days_available_in_period': days_available,
        'usage_frequency': days_used / pd.Series(days_available, index=days_used.index)
    }, index=user_registration.index).fillna(0)

    # Session Pattern Features
    session_lengths = df.groupby(['userId', 'sessionId'])['itemInSession'].max()
    session_length_stats = session_lengths.groupby('userId').agg(['mean', 'std', 'max']).fillna(0)
    session_length_stats.columns = ['avg_session_length', 'session_length_std', 'max_session_length']

    session_durations = df.groupby(['userId', 'sessionId'])['ts'].apply(
        lambda x: (x.max() - x.min()).total_seconds() / 60
    )
    session_duration_stats = session_durations.groupby('userId').agg(['mean', 'std', 'max']).fillna(0)
    session_duration_stats.columns = ['avg_session_duration_mins', 'session_duration_std_mins', 'max_session_duration_mins']

    session_features = pd.concat([session_length_stats, session_duration_stats], axis=1)
    session_features['session_consistency'] = 1 / (1 + session_features['session_length_std'])

    # Initialize feature dataframe
    all_users = df['userId'].dropna().unique()
    final_features = pd.DataFrame(index=all_users)

    # List of all feature DataFrames
    feature_sets = [
        demographic_final,       # Demographic features
        user_activity,           # Activity features
        listening_features,      # Listening features  
        engagement_features,     # Engagement features
        subscription_features,   # Subscription features
        issue_features,          # Problem/issue features
        temporal_features,       # Temporal features
        session_features         # Session features
    ]

    # Merge all feature sets
    for i, feature_set in enumerate(feature_sets):
        final_features = final_features.join(feature_set, how='left')

    # Fill any remaining NaNs with 0
    final_features = final_features.fillna(0)
    churned_users = df.query("page == 'Cancellation Confirmation'")['userId'].unique()
    final_features['is_churned'] = final_features.index.isin(churned_users).astype(int)
    print(f"Final feature matrix shape: {final_features.shape}")
    print(f"Features created: {len(final_features.columns)-1}")
    print(f"Churn rate in final dataset: {final_features['is_churned'].mean():.1%}")
    return final_features

In [18]:
features_large = extract_features(dataframe_processed)

Final feature matrix shape: (448, 47)
Features created: 46
Churn rate in final dataset: 22.1%


In [20]:
# Load your trained model
model = joblib.load('../models/lg_churn.pkl')  # Save your model first

# Make predictions
X_large = features_large.drop('is_churned', axis=1)
y_large = features_large['is_churned']

print(f"\nValidating on {len(X_large):,} users...")
y_pred = model.predict(X_large)

# Evaluate
print("\nValidation Results on Large Dataset:")
print(classification_report(y_large, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_large, y_pred))


Validating on 448 users...

Validation Results on Large Dataset:
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       349
           1       0.69      0.73      0.71        99

    accuracy                           0.87       448
   macro avg       0.81      0.82      0.81       448
weighted avg       0.87      0.87      0.87       448


Confusion Matrix:
[[317  32]
 [ 27  72]]


In [21]:
features_large

,gender_F,gender_M,state_CA,state_NY-NJ-PA,state_Other,state_TX,total_events,num_sessions,total_interactions,events_per_session,songs_played,total_listening_time,avg_song_length,unique_artists,unique_songs,days_since_registration,avg_daily_listening_time,avg_daily_songs,artist_diversity,thumbs_up,thumbs_down,total_feedback,positive_feedback_ratio,playlist_adds,add_friend,advert_roll,is_paid,subscription_changes,downgrades,upgrades,error_count,help_visits,settings_visits,logout_count,has_issues,days_since_last_activity,days_used_in_period,days_available_in_period,usage_frequency,avg_session_length,session_length_std,max_session_length,avg_session_duration_mins,session_duration_std_mins,max_session_duration_mins,session_consistency,is_churned
293,False,True,False,False,False,True,5284,58,689968,91.103448,4246,1.049255e+06,247.116034,2324,3473,133.784248,7842.886574,31.737668,0.547339,227.0,39.0,266.0,0.853383,103.0,84.0,51.0,1,1,1.0,2.0,5.0,27.0,26.0,62.0,1.0,9.184456,42,135,0.311111,90.448276,125.696540,688,297.813506,425.999175,2336.566667,0.007893,1
98,False,True,False,False,False,True,3290,29,379279,113.448276,2687,6.720821e+05,250.123580,1678,2296,64.268843,10457.354336,41.808750,0.624488,131.0,31.0,162.0,0.808642,75.0,39.0,7.0,1,1,0.0,1.0,7.0,21.0,18.0,34.0,1.0,0.321181,33,66,0.500000,115.000000,116.051097,438,392.292529,421.212572,1569.233333,0.008543,0
179,False,True,False,False,True,False,2716,20,362263,135.800000,2219,5.520414e+05,248.779381,1428,1920,88.761701,6219.365310,24.999521,0.643533,94.0,19.0,113.0,0.831858,59.0,45.0,4.0,1,0,0.0,0.0,0.0,20.0,17.0,30.0,1.0,0.211377,21,90,0.233333,144.000000,133.888799,457,455.774167,432.981695,1584.416667,0.007414,0
246,True,False,False,False,True,False,3656,23,753071,158.956522,2999,7.545518e+05,251.601132,1802,2546,89.021389,8476.073061,33.688533,0.600867,153.0,34.0,187.0,0.818182,80.0,39.0,23.0,1,1,1.0,1.0,5.0,18.0,24.0,38.0,1.0,26.019468,25,91,0.274725,158.913043,204.730624,925,542.094203,729.250155,3298.566667,0.004861,1
163,True,False,False,False,True,False,6120,36,1053716,170.000000,5050,1.252230e+06,247.966242,2635,4072,120.911528,10356.576779,41.766076,0.521782,242.0,35.0,277.0,0.873646,149.0,81.0,4.0,1,0,0.0,0.0,8.0,16.0,30.0,65.0,1.0,0.000718,42,122,0.344262,176.388889,173.328965,658,598.008796,613.015377,2343.716667,0.005736,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200031,False,True,False,False,True,False,44,1,946,44.000000,27,6.578326e+03,243.641693,27,27,171.170058,38.431521,0.157738,1.000000,2.0,2.0,4.0,0.500000,1.0,1.0,3.0,0,0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,7.365880,1,173,0.005780,43.000000,0.000000,43,105.316667,0.000000,105.316667,1.000000,0
305,False,True,False,False,True,False,134,1,8911,134.000000,100,2.500687e+04,250.068705,97,99,6.349444,3938.434419,15.749409,0.970000,6.0,0.0,6.0,1.000000,1.0,5.0,10.0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.744988,2,8,0.250000,133.000000,0.000000,133,417.700000,0.000000,417.700000,1.000000,0
266,True,False,False,False,False,True,37,1,666,37.000000,22,5.557123e+03,252.596503,22,22,89.196910,62.301744,0.246645,1.000000,2.0,0.0,2.0,1.000000,1.0,2.0,2.0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,4.194236,1,91,0.010989,36.000000,0.000000,36,89.733333,0.000000,89.733333,1.000000,0
200012,False,True,False,False,True,False,50,1,1225,50.000000,41,1.129867e+04,275.577366,40,41,65.777894,171.770049,0.623310,0.975610,1.0,1.0,2.0,0.500000,3.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.704699,1,67,0.014925,49.000000,0.000000,49,184.716667,0.000000,184.716667,1.000000,0


In [29]:
features_large.reset_index().rename(columns={"index": "user_id"}).drop(columns=["is_churned"]).to_json("../src/data/user_features.json")